In [1]:
# Notebook for iterative inference (multi-step) on ARTRFDC models

In [2]:
import os
import json
import numpy as np
import pandas as pd
import joblib
from datetime import datetime
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error

# uncomment the following line for compatibility with TensorFlow 1.15 (on GCP)
# import tensorflow.compat.v1 as tf
# uncomment the following line for TensorFlow 2.X (local execution)
import tensorflow as tf

# forecast model was saved in TensorFlow 1.15
# but, in order to make predictions locally, has to be loaded with TensorFlow 2
from tensorflow.saved_model import load

In [3]:
from bokeh.plotting import figure, show, output_file, save
from bokeh.io import output_notebook
# select a palette
from bokeh.palettes import d3
output_notebook()

Loading BokehJS ...

In [4]:
# symmetrical mean absolute percentage error
def smape(targets, predictions):
    '''
    predictions: a list with the predicted values
    targets: a list with the actual values
    '''
    import numpy as np
    # lists to NumPy arrays
    targets, predictions = np.array(targets), np.array(predictions)
    # verify predictions and targets have the same shape
    if predictions.shape == targets.shape:
            return(np.sum(2*np.abs(predictions - targets) /
                          (np.abs(targets) + np.abs(predictions)))/predictions.shape[0])

In [5]:
# for iterative inference on an ARTRFDC model, all information in SLDB should be parsed
# therefore this version of parsing function is different from the one in the train-eval loop
def _parse_full_dataset_function(example_proto, objective_shapes):
    # parse the input tf.Example proto using the dictionary above
    row = tf.io.parse_single_example(example_proto, read_features)
    
    # pass objective shape as a list of lists
    source = tf.reshape(row['source'].values, objective_shapes['source'])
    target = tf.reshape(row['target'].values, objective_shapes['target'])
    target_pos_encoding = tf.reshape(row['target_pos_encoding'].values, objective_shapes['target_pos_encoding'])
    source_timestamp = tf.reshape(row['source_timestamp'].values, objective_shapes['source_timestamp'])
    target_timestamp = tf.reshape(row['target_timestamp'].values, objective_shapes['target_timestamp'])


    # the parsed dataset have now the shape {features}, {labels}
    # so:
    features_dict = {
        'source': source,
        'target_pos_encoding': target_pos_encoding,
        'source_timestamp': source_timestamp,
        'target_timestamp': target_timestamp
    }
        
    # _parse_dataset_function returns:
    # features_dict as model features
    # target as model labels
    return features_dict, target

In [6]:
# a function to encode float values for serialized examples
def _float_feature_from_list_of_values(list_of_values):
    """Returns a float_list from a list of floats / doubles."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=list_of_values))

In [7]:
def tensor_to_tensor_example(float_tensor):
    # first, pass the float tensor to NumPy array, then flatten it
    flat_array = float_tensor.numpy().flatten()
    # second, build the protobuffer example
    example = tf.train.Example(
        # features within the example
        features=tf.train.Features(
            # individual feature definition
            feature={'source': _float_feature_from_list_of_values(flat_array)}
        )
    )    
    # third, serialize the example dictionary to a string
    serialized_example = example.SerializeToString()
    # fourth, wrap the serialized example as a NumPy-string array
    numpy_example = np.array(serialized_example, dtype='S')
    # fifth, wrap the NumPy-string array as a string tensor
    tensor_example = tf.convert_to_tensor(numpy_example)

    return tensor_example

In [9]:
read_features = {
    'source': tf.io.VarLenFeature(dtype=tf.float32),
    'target': tf.io.VarLenFeature(dtype=tf.float32),
    'target_pos_encoding': tf.io.VarLenFeature(dtype=tf.float32),
    'source_timestamp': tf.io.VarLenFeature(dtype=tf.string),
    'target_timestamp': tf.io.VarLenFeature(dtype=tf.string)
}

In [10]:
PROJECT_ROOT = '/home/developer/gcp/cbidmltsf'

In [11]:
# during batch prediction, the model identifier is obtained via Abseil Flags
# remember this notebook is based on local execution,
# therefore model directory must be downloaded from GS before running the notebook
model_id = 'ARTRFDC_TPU_001'

# during batch prediction, the SLDB identifier is obtained via Abseil Flags
# THE SLDB FOR INFERENCE MUST BE THE SAME USED FOR TRAINING! (THE ONE SETUP IN THE CONFIGURATION FILE)
sldb_id = 'CPE04115_H_kw_20210526212214_ARTRFDC_168'

# during batch prediction, the dataset name is obtained via Abseil Flags
dataset = 'test'

# define a forecast window to guide the iterative prediction process
# start with a hourly, day-ahead process
forecast_window = 24

# ADD AN INFERENCE IDENTIFIER, BECAUSE FOR ARTRFDC MODELS, DIFFERENT INFERENCES
# CAN BE PRODUCED FROM A SINGLE SAVED MODEL (USUALLY DIFFERENT FORECAST WINDOWS)
# during batch prediction, the inference identifier should be obtained via Abseil Flags
inference = '024'

In [12]:
# build a path to the SLDB json file
data_dir = '{}/{}/{}'.format(PROJECT_ROOT, 'sldbs', sldb_id)

# then get the ts_identifier from the json file in the sldb directory
sldb_json_file = '{}/sldb.json'.format(data_dir)

In [13]:
# open the json file
with open(sldb_json_file, 'r') as inputfile:
    sldb_dict = json.load(inputfile)

In [14]:
# and get the time series identifier
ts_identifier = sldb_dict['ts']

In [15]:
# use the time series identifier to obtain the SK-Learn scaler used on it
# get the scaler used to normalize the test dataset (unseen)
scaler_test = joblib.load('{}/{}/{}/scaler_test.save'.format(PROJECT_ROOT,
                                                             'timeseries',
                                                             ts_identifier))

print('Scaler on test dataset loaded for time series {}'.format(ts_identifier))

Scaler on test dataset loaded for time series CPE04115_H_kw_20210526212214


In [16]:
# pass all the code to a single notebook cell, then to a function, later...

In [17]:
# get the SLDB parameters for the forecasting model
config_json_file = '{}/{}/{}.json'.format(PROJECT_ROOT,
                                          'parameters',
                                          model_id)

# recover the sldb dictionary from the json file in parameters/
with open(config_json_file, 'r') as inputfile:
    configuration = json.load(inputfile)

In [18]:
# store the objective shapes for reshaping tensors in a dictionary
_EXTRACTING_OBJECTIVE_SHAPES = {
    'source': [configuration['num_timesteps'], configuration['model_dimension']],
    'target': [configuration['num_timesteps'], 1],
    'target_pos_encoding': [configuration['num_timesteps'], 6],
    'source_timestamp': [configuration['num_timesteps'], 1],
    'target_timestamp': [configuration['num_timesteps'], 1]
}

In [19]:
# build a path to the dataset for prediction
dataset_path = '{}/{}.tfrecord'.format(data_dir, dataset)

# load the dataset
tfrecord_dataset = tf.data.TFRecordDataset(dataset_path)

In [20]:
# parse the test dataset from the TFRecord file
# and use its features to produce predictions in a different way

parsed_dataset = tfrecord_dataset.map(
    lambda row: _parse_full_dataset_function(
        example_proto=row,
        objective_shapes=_EXTRACTING_OBJECTIVE_SHAPES
    )
)

In [21]:
# build temporary lists to store source (features) and target (labels) tensors
source_list = list()
source_timestamp_list = list()

target_list = list()
target_pos_encoding_list = list()
target_timestamp_list = list()

# fill in the lists from the parsed dataset
for element in parsed_dataset:
    source_list.append(element[0]['source'])
    # convert timestamps from bytes to strings to flat array to datetime
    source_timestamp_list.append(pd.to_datetime(element[0]['source_timestamp'].numpy().astype(str).squeeze()))
    
    target_list.append(element[1])
    target_pos_encoding_list.append(element[0]['target_pos_encoding'])
    # convert timestamps from bytes to strings to flat array to datetime
    target_timestamp_list.append(pd.to_datetime(element[0]['target_timestamp'].numpy().astype(str).squeeze()))

# from the temporary lists, build NumPy arrays to feed the model
source_array = np.array(source_list)
source_timestamp_array = np.array(source_timestamp_list)

target_array = np.array(target_list)
target_pos_encoding_array = np.array(target_pos_encoding_list)
target_timestamp_array = np.array(target_timestamp_list)

In [22]:
# from now on, inferences for ARTRFDC are produced in a very different way
# from the used for DMSLSTM or EDALSTM models (prediction process has to be iterative)

In [23]:
# each predicted row is a sequence of n_timesteps values,
# but only the first element in this sequence is used, as the first prediction,
# then it is added (with its positional encoding) to the end of the input sequence
# (first entry of the input sequence is discarded to keep tensor shape)
# to get the second prediction, and so on up to the n_timesteps-th prediction,
# which completes the n_timesteps prediction sequence (the forecast window)
# that starts immediately after the source input sequence ends (in time dimension)

In [24]:
# the iterative process for inference over the ARTRFDC saved model can be initiated now:
# source feature (?, 168, 7) (values and pos encoding, unseen data) is on parsed_dataset[0]['source']
# target feature (?, 168, 1) (values only, unseen data) is on parsed_dataset[1]

# uncomment and run the following two cells to confirm that

In [25]:
# now follow the inference process detailed in Klingenbrunn to:
# predict over the forecast window,
# calculate prediction error metrics, and
# plot prediction results

In [26]:
# ToDo: adjust previous code to new SLDB
# target was [168, 7] and now is [168, 1]
# while the missing features are now in target_pos_encoding [168, 6]

In [27]:
# in the source array, the index of the last row that can be used
# to build a forecast-window sized set of predictions
max_row_index = len(source_list) - configuration['num_timesteps'] - 1

print('Given the current dataset and the number of timesteps in the input sequence,')
print('predictions can be made between row 0 and row {} of the source array.'.format(max_row_index))

Given the current dataset and the number of timesteps in the input sequence,
predictions can be made between row 0 and row 1871 of the source array.


In [28]:
# the first source or input to the model is the first source row
# that means, the true variable value, plus the six positional encodings for the timestamp
# in the first row of the test dataset

In [29]:
# uncomment the following line to get the source as a tensor with TensorShape([1, 168, 7])
# source_tensor = tf.expand_dims(source_array[0, :, :], axis=0)

In [30]:
# important, the inference cycle was coded for tensors, not for NumPy arrays
# then use source and prediction tensors and translate to tensor examples from float tensors

### run inference process and build databases

In [31]:
# during batch prediction, the execution identifier is obtained via Abseil Flags
for execution in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]:
    
    # a columns list for the predictions dataframe
    pred_df_columns = ['model_id',
                       'execution',
                       'dataset',
                       'inference',
                       'string_timestamps',
                       'predictions',
                       'targets']
    
    predictions_detail_df = pd.DataFrame(columns=pred_df_columns)

    # use model identifier and execution number to build the model directory string
    model_dir = '{}_{:02d}'.format(model_id, execution)

    # get the path to the saved model main directory
    saved_model_path = '{}/{}/{}/export/exporter'.format(PROJECT_ROOT,
                                                         'models',
                                                         model_dir)

    # get all the files in the saved model path, to find the most recent one
    all_files = os.listdir(saved_model_path)
    # get the path to the most recent saved model
    latest_saved_model_id = sorted(all_files)[-1]

    # build the full path for the latest saved model dir
    export_dir = '{}/{}'.format(saved_model_path, latest_saved_model_id)
    print ('Exported model path is {}'.format(export_dir))

    # load the saved model and the prediction function
    imported = load(export_dir=export_dir, tags='serve')
    predict_fn = imported.signatures["serving_default"]
    
    # iterate on the valid rows of the test dataset
    for row in np.arange(max_row_index + 1):

        # then again, build the initial source tensor
        source_tensor = tf.expand_dims(source_array[row, :, :], axis=0)

        # and build the initial prediction tensor
        # a forecast-window-sized tensor (1, forecast_window, 7)
        # formed with the forecast_window true values, starting at the end of the source tensor
        # that means

        prediction_tensor = tf.expand_dims(
            source_array[row + configuration['num_timesteps'], :forecast_window, :],
            axis=0)

        # remember source_array is (?, timesteps, features), but source_timestamp_array is (?, timesteps) only
        prediction_timestamps = source_timestamp_array[row + configuration['num_timesteps'], :forecast_window]


        next_input_model = source_tensor

        # re-initialize the prediction list previously used for prediction over TFRecords
        predictions_list = list()

        # iterate on the forecast window timesteps for the current row of the test dataset
        # fill the predictions list over the forecast window
        for i in range(forecast_window):

            # from the current next_input_model tensor (1, 168, 7)
            # get a prediction as NumPy array (1, 168, 1)
            prediction = predict_fn(tensor_to_tensor_example(next_input_model))['forecast'].numpy()

            # get the value of the most recent prediction (last timestep) into the predictions list
            predictions_list.append(prediction[:, -1, :][0][0])

            # from the source tensor, get the positional encodings for ti+1 to t167 (that is 168-i-1 values)
            pos_encoding_old_values = source_tensor[:, i+1:, 1:]

            # from target tensor, get the positional encodings for t168 to t168+i (that is i+1 values)
            pos_encoding_new_val = prediction_tensor[:, :i+1, 1:]

            # build new positional encodings with 168 values
            pos_encodings = tf.concat([pos_encoding_old_values, pos_encoding_new_val], axis=1)
            pos_encodings = tf.cast(pos_encodings, dtype=tf.float32)

            # build the values feature for the next input to the model
            # pop i+1 values at the beginning of the previous input
            value_feature_old_values = tf.expand_dims(source_tensor[:, i+1:, 0], axis=-1)
            value_feature_old_values = tf.cast(value_feature_old_values, dtype=tf.float32)

            # current predictions_list to NumPy array
            value_feature_new_values = np.array(predictions_list[:i+1])

            # current prediction array to tensor
            value_feature_new_values = tf.convert_to_tensor(value_feature_new_values)

            # expand dimensions of current prediction tensor to single-value feature
            value_feature_new_values = tf.expand_dims(value_feature_new_values, axis=-1)

            # expand dimensions of current prediction tensor to single-value batch
            value_feature_new_values = tf.expand_dims(value_feature_new_values, axis=0)

            # build the value feature tensor
            next_input_model = tf.concat([value_feature_old_values, value_feature_new_values], axis=1)

            # build the next input tensor for the model
            next_input_model = tf.concat([next_input_model, pos_encodings], axis=2)


        # iterative predictions over the forecast window reside in predictions_list
        # convert list to array, then expand feature dimension with value 1
        predicted_values = np.array(predictions_list).reshape(-1, 1)

        # inverse-scale predictions
        rescaled_predicted_values = scaler_test.inverse_transform(predicted_values)

        # and the true values remain in the prediction tensor, pass them to a NumPy array
        # for the true values array, expand feature dimension with value 1
        true_values = prediction_tensor[0, :, 0].numpy().reshape(-1, 1)

        # inverse-scale true values
        rescaled_true_values = scaler_test.inverse_transform(true_values)

        # a temporary dataframe built from the data in the current row
        df = pd.DataFrame(columns=pred_df_columns)
        df['model_id'] = [model_id]
        df['execution'] = [execution]
        df['dataset'] = [dataset]
        df['inference'] = [inference]
        df['string_timestamps']= [pd.to_datetime(prediction_timestamps).astype(str).tolist()]
        df['predictions'] = [np.squeeze(rescaled_predicted_values).tolist()]
        df['targets'] = [np.squeeze(rescaled_true_values).tolist()]
        df['mae'] = mean_absolute_error(rescaled_true_values, rescaled_predicted_values)
        df['rmse'] = sqrt(mean_squared_error(rescaled_true_values, rescaled_predicted_values))
        df['smape'] = smape(rescaled_true_values, rescaled_predicted_values)

        # append the temporary dataframe to the predictions detail dataframe
        predictions_detail_df = pd.concat([predictions_detail_df, df])

    
    # reset the index of final dataframe, once all of its rows (dataset) have been processed
    predictions_detail_df = predictions_detail_df.reset_index(drop=True)

    # build a predictions summary dataframe, reset index to avoid making a multi-column index when grouping by
    predictions_summary_df = predictions_detail_df.groupby(['model_id',
                                                            'execution',
                                                            'dataset',
                                                            'inference']).mean().reset_index()

    # a range to iterate on prediction timesteps
    targets_range = np.arange(forecast_window)

    # vector metric (vector component to vector component)
    # an array forecast_window-d: metric for 1, 2,..., no_targets step-ahead
    # (target versus prediction for rows in dataset)

    # for index, row in dataframe.iterrows()
    mae_vector = [
        mean_absolute_error(
            # a list with the n-rows target values for the n-th step ahead
            [row.targets[n] for _, row in predictions_detail_df.iterrows()],
            # a list with the n-rows prediction values for the n-th step ahead
            [row.predictions[n] for _, row in predictions_detail_df.iterrows()]
        ) for n in targets_range
    ]
    predictions_summary_df['mae_vector'] = [mae_vector]

    # for index, row in dataframe.iterrows()
    rmse_vector = [
        sqrt(mean_squared_error(
            # a list with the n-rows target values for the n-th step ahead
            [row.targets[n] for _, row in predictions_detail_df.iterrows()],
            # a list with the n-rows prediction values for the n-th step ahead
            [row.predictions[n] for _, row in predictions_detail_df.iterrows()]
        )) for n in targets_range
    ]
    predictions_summary_df['rmse_vector'] = [rmse_vector]
    
    # for index, row in dataframe.iterrows()
    smape_vector = [
        smape(
            [row.targets[n] for _, row in predictions_detail_df.iterrows()],
            [row.predictions[n] for _, row in predictions_detail_df.iterrows()]
        ) for n in targets_range
    ]
    predictions_summary_df['smape_vector'] = [smape_vector]

    # insert count of rows as a column value
    predictions_summary_df.insert(4, 'count', max_row_index + 1)

    # build a path to persist the dataframe to database/predictions_detail/
    detail_pickle_path = '{}/{}/{}/{}_{:02d}_{}_{}.pkl'.format(
        PROJECT_ROOT,
        'database',
        'predictions_detail',
        model_id,
        execution,
        dataset,
        inference)
    
    # persist the Pandas dataframe to database/predictions_detail/
    predictions_detail_df.to_pickle(detail_pickle_path)
    print('Persisted Pandas dataframe for predictions detail of {}_{:02d}_{}_{}'.format(model_id,
                                                                                        execution,
                                                                                        dataset,
                                                                                        inference))

    # build a path to persist the dataframe to database/predictions_summary/
    summary_pickle_path = '{}/{}/{}/{}_{:02d}_{}_{}.pkl'.format(
        PROJECT_ROOT,
        'database',
        'predictions_summary',
        model_id,
        execution,
        dataset,
        inference)

    # persist the Pandas dataframe to database/predictions_summary/
    predictions_summary_df.to_pickle(summary_pickle_path)
    print('Persisted Pandas dataframe for predictions summary of {}_{:02d}_{}_{}'.format(model_id,
                                                                                         execution,
                                                                                         dataset,
                                                                                         inference))


Exported model path is /home/developer/gcp/cbidmltsf/models/ARTRFDC_TPU_001_00/export/exporter/1622675695
Persisted Pandas dataframe for predictions detail of ARTRFDC_TPU_001_00_test_024
Persisted Pandas dataframe for predictions summary of ARTRFDC_TPU_001_00_test_024
Exported model path is /home/developer/gcp/cbidmltsf/models/ARTRFDC_TPU_001_01/export/exporter/1622676190
Persisted Pandas dataframe for predictions detail of ARTRFDC_TPU_001_01_test_024
Persisted Pandas dataframe for predictions summary of ARTRFDC_TPU_001_01_test_024
Exported model path is /home/developer/gcp/cbidmltsf/models/ARTRFDC_TPU_001_02/export/exporter/1622676387
Persisted Pandas dataframe for predictions detail of ARTRFDC_TPU_001_02_test_024
Persisted Pandas dataframe for predictions summary of ARTRFDC_TPU_001_02_test_024
Exported model path is /home/developer/gcp/cbidmltsf/models/ARTRFDC_TPU_001_03/export/exporter/1622676597
Persisted Pandas dataframe for predictions detail of ARTRFDC_TPU_001_03_test_024
Persis

In [34]:
# ToDo: adjust the following code to include the new column 'inference'

In [20]:
# to plot results, get the required predictions detail dataframe from a pickle file

In [26]:
model_id, execution, dataset = 'ARTRFDC_TPU_001', 9, 'test'

In [27]:
# build a path to persist the dataframe to database/predictions_detail/
detail_pickle_path = '{}/{}/{}/{}_{:02d}_on_{}_tfrecord.pkl'.format(
    PROJECT_ROOT,
    'database',
    'predictions_detail',
    model_id,
    execution,
    dataset)

In [28]:
predictions_detail_df = pd.read_pickle(detail_pickle_path)

In [38]:
# how does behave SMAPE on forecast_window-th-step-ahead predictions, over the test dataset
# (as predictions go further from the training and evaluation datasets periods)

In [39]:
# a dictionary to manage plots
plots = dict()

In [72]:
plots['smape'] = figure(
    # x_axis_type='datetime',
    plot_width=960,
    plot_height=400,
    title='SMAPE for model {}, execution {}, on {} dataset'.format(
        model_id,
        execution,
        dataset))

plots['smape'].grid.grid_line_alpha=0.5

plots['smape'].xaxis.axis_label = 'Row of test dataset'
plots['smape'].yaxis.axis_label = 'SMAPE for the forecast window'

plots['smape'].line(
    np.arange(max_row_index + 1),
    predictions_detail_df['smape'],
    color='red',
    legend_label='SMAPE')

# uncomment the following two lines to save plot
# output_file('/home/developer/gcp/cbidmltsf/datasets/cfe/{}_H_kw.html'.format(device))
# save(fig_kw)

# uncomment the following line to display plot
show(plots['smape'])

In [74]:
# repeat the previous plot, replace row ordinal with its start timestamp

In [86]:
# 1-d dataframe contains 1-d lists, change it to 2-d array
# for index, row in dataframe.iterrows():
temporary_list = list()
for _, row in predictions_detail_df.iterrows():
    # append the first timestamp in 'string_timestamps to the temporary list'
    temporary_list.append(pd.to_datetime(row['string_timestamps'][0]))

In [88]:
first_timestamps_array = np.array(temporary_list)

In [96]:
# VERY IMPORTANT!:due to the transformer nature (iterative along a given num_timesteps),
# given the number of timesteps of the model and the starting timestamp of the data for inference,
# the first prediction that can be made starts num_timesteps beyond the inference starting timestamp

In [98]:
# for the present experiment, num_timesteps is:
configuration['num_timesteps']

168

In [99]:
# and the starting timestamp for inference data is
source_timestamp_list[0][0]

Timestamp('2018-05-01 00:00:00')

In [100]:
# then the first prediction can be made on:
first_timestamps_array[0]

Timestamp('2018-05-08 00:00:00')

In [113]:
# redefine the previous plot for a given number of days and not for the complete test dataset
num_days_ahead_for_inference = 1
num_selected_rows = 24*num_days_ahead_for_inference

In [114]:
plots['smape_timestamp'] = figure(
    x_axis_type='datetime',
    plot_width=960,
    plot_height=400,
    title='SMAPE for model {}, execution {}, on {} dataset'.format(
        model_id,
        execution,
        dataset))

plots['smape_timestamp'].grid.grid_line_alpha=0.5

plots['smape_timestamp'].xaxis.axis_label = 'Forecast window initial timestamp'
plots['smape_timestamp'].yaxis.axis_label = 'SMAPE for the forecast window'

plots['smape_timestamp'].line(
    first_timestamps_array[:num_selected_rows],
    predictions_detail_df['smape'][:num_selected_rows],
    color='red',
    legend_label='SMAPE')

# uncomment the following two lines to save plot
# output_file('/home/developer/gcp/cbidmltsf/datasets/cfe/{}_H_kw.html'.format(device))
# save(fig_kw)

# uncomment the following line to display plot
show(plots['smape_timestamp'])

In [73]:
# plot predictions for a single row in the test dataset
row = 296

plots['identifier'] = figure(
    x_axis_type='datetime',
    plot_width=960,
    plot_height=400,
    title='Multi-step prediction for model {}, execution {}, on row {} of {} dataset, SMAPE = {}'.format(
        model_id,
        execution,
        row,
        dataset,
        predictions_detail_df.loc[row, 'smape']))

plots['identifier'].grid.grid_line_alpha=0.5

plots['identifier'].xaxis.axis_label = 'Timestamp'
plots['identifier'].yaxis.axis_label = 'Active Power [KW]'

plots['identifier'].line(
    pd.to_datetime(predictions_detail_df.loc[row, 'string_timestamps']),
    predictions_detail_df.loc[row, 'predictions'],
    color='green',
    legend_label='real')

plots['identifier'].line(
    pd.to_datetime(predictions_detail_df.loc[row, 'string_timestamps']),
    predictions_detail_df.loc[row, 'targets'],
    color='blue',
    legend_label='predicted')

# uncomment the following two lines to save plot
# output_file('/home/developer/gcp/cbidmltsf/datasets/cfe/{}_H_kw.html'.format(device))
# save(fig_kw)

# uncomment the following line to display plot
show(plots['identifier'])